In [13]:
%load_ext autoreload
%autoreload 2

In [14]:
import sys
sys.path.append("../")
# !conda install --yes --prefix {sys.prefix} gensim

In [15]:
# Setting up N2VEC parameters 
from topsy_turvy.comparison_methods.n2vec_embedding import compute_embedding


test_file   = "/afs/csail.mit.edu/u/k/kdevko01/Research/Topsy-Turvy/datasets/Analysis/flydata/data/all_nodepairs.tsv"
input_folder  = "/afs/csail.mit.edu/u/k/kdevko01/Research/Topsy-Turvy/datasets/Analysis/flydata/data/subnets/"
output_folder = "/afs/csail.mit.edu/u/k/kdevko01/Research/Topsy-Turvy/datasets/Analysis/flydata/data/subnets/"
networks      = ["network_0.2", "network_0.4", "network_0.6", "network_0.8"]

In [19]:
df = pd.read_csv(test_file, sep = "\t")
# Edges
(df["True_score"] > 0.5).sum()
# Nodes
len(set(df["node1"]).union(set(df["node2"])))

3093

In [46]:
import pandas as pd
import json
# Given a network file 
for network in networks:
    print(f"Processing network {input_folder}/{network}")
    net_file       = f"{input_folder}/{network}.tsv"
    json_file      = f"{output_folder}/n2vec_{network}.json"
    df    = pd.read_csv(net_file, sep = "\t", header = None)
    df.columns = ["p", "q"]
    nodes     = set(df["p"]).union(set(df["q"]))
    nodes_map = {node: i for i, node in enumerate(nodes)}
    with open(json_file, "w") as jf:
        json.dump(nodes_map, jf)
    df_annotated           = df.replace({"p": nodes_map, "q": nodes_map})
    df_annotated["weight"] = 1
    df_annotated.to_csv(f"{output_folder}/n2vec_annotated_{network}.tsv", 
                        header = False, 
                        index = False, 
                        sep = "\t")
df_annotated

Processing network /afs/csail.mit.edu/u/k/kdevko01/Research/Topsy-Turvy/datasets/Analysis/flydata/data/subnets//network_0.2
Processing network /afs/csail.mit.edu/u/k/kdevko01/Research/Topsy-Turvy/datasets/Analysis/flydata/data/subnets//network_0.4
Processing network /afs/csail.mit.edu/u/k/kdevko01/Research/Topsy-Turvy/datasets/Analysis/flydata/data/subnets//network_0.6
Processing network /afs/csail.mit.edu/u/k/kdevko01/Research/Topsy-Turvy/datasets/Analysis/flydata/data/subnets//network_0.8


,p,q,weight
0,140,2083,1
1,140,1960,1
2,140,575,1
3,140,2485,1
4,140,835,1
...,...,...,...
5421,2319,602,1
5422,1224,2779,1
5423,2024,2393,1
5424,482,184,1


In [38]:
# NODE2VEC ARGS
args      = {}
args["p"] = 1
args["q"] = 1
args["dimensions"] = 300
args["num_walks"]  = 10
args["intermediate_file_loc"] = f"{output_folder}/{network}_text_dim_{args['dimensions']}.emb"
args["final_emb"]             = f"{output_folder}/{network}_emb_dim_{args['dimensions']}.npy"

In [16]:
df_annotated = pd.read_csv(f"{output_folder}/annotate_{network}.tsv", sep = "\t", header = None)
df_annotated

,0,1,2
0,163,2349,0.898801
1,3283,1459,0.004207
2,1725,1565,0.394838
3,455,2894,0.004177
4,245,19,0.004201
...,...,...,...
27465,351,6,0.004178
27466,3420,1722,0.987240
27467,1559,1386,0.985707
27468,2287,1202,0.004253


In [17]:
edgelist = list(df_annotated.to_records(index = False))

In [50]:
args      = {}
args["p"] = 1
args["q"] = 1
args["dimensions"] = 300
args["num_walks"]  = 10

def save_emb(args):
    with open(args["intermediate_file_loc"], "r") as ef:
        n_nodes, n_emb = ef.readline().strip().split(" ")
        n_nodes        = int(n_nodes)
        n_emb          = int(n_emb)
        embeddings     = np.zeros((n_nodes, n_emb))
    
        for line in ef:
            emb        = line.strip().split(" ")
            index      = int(emb[0])
            features   = [float(f_i) for f_i in emb[1:]]
            embeddings[index] = features
    np.save(args["final_emb"], embeddings)

for network in networks:
    print(f"Computing the embedding for the network {network}")
    df_annotated = pd.read_csv(f"{output_folder}/n2vec_annotated_{network}.tsv", sep = "\t", header = None)
    edgelist = list(df_annotated.to_records(index = False))
    args["intermediate_file_loc"] = f"{output_folder}/n2vec_{network}_text_dim_{args['dimensions']}.emb"
    args["final_emb"]             = f"{output_folder}/n2vec_{network}_emb_dim_{args['dimensions']}.npy"
    embeddings = compute_embedding(edgelist, args)
    print(f"Saving...")
    save_emb(args)

Computing the embedding for the network network_0.2
Walk iteration:
1 / 10
2 / 10
3 / 10
4 / 10
5 / 10
6 / 10
7 / 10
8 / 10
9 / 10
10 / 10
{'p': 1, 'q': 1, 'dimensions': 300, 'num_walks': 10, 'intermediate_file_loc': '/afs/csail.mit.edu/u/k/kdevko01/Research/Topsy-Turvy/datasets/Analysis/flydata/data/subnets//n2vec_network_0.2_text_dim_300.emb', 'final_emb': '/afs/csail.mit.edu/u/k/kdevko01/Research/Topsy-Turvy/datasets/Analysis/flydata/data/subnets//n2vec_network_0.2_emb_dim_300.npy', 'input': 'karate.edgelist', 'walk-length': 80, 'num-walks': 10, 'window-size': 10, 'iter': 1, 'workers': 8, 'weighted': True, 'unweighted': False, 'directed': False, 'undirected': True}
Here
Saving...
Computing the embedding for the network network_0.4
Walk iteration:
1 / 10
2 / 10
3 / 10
4 / 10
5 / 10
6 / 10
7 / 10
8 / 10
9 / 10
10 / 10
{'p': 1, 'q': 1, 'dimensions': 300, 'num_walks': 10, 'intermediate_file_loc': '/afs/csail.mit.edu/u/k/kdevko01/Research/Topsy-Turvy/datasets/Analysis/flydata/data/subnet

In [3]:
from sklearn.linear_model import LogisticRegression
import pandas as pd
def construct_and_train_model(df, emb):
    def compute_neg_set(pos_edges, n_nodes, n_negatives):
        neg_set = []
        while(len(neg_set) < n_negatives):
            p = np.random.randint(n_nodes)
            q = np.random.randint(n_nodes)
            if p > q:
                temp = p
                p    = q 
                q    = temp
            if p == q:
                continue
            if f"{p}_{q}" in pos_edges:
                continue
            neg_set.append((p, q))
        return neg_set
    
    def construct_hadamard(edgelists, n_dim):
        features = np.zeros((len(edgelists), n_dim))
        for i, (p, q) in enumerate(edgelists):
            e1 = emb[p]
            e2 = emb[q]
            features[i] = e1 * e2
        return features
            
        
    df.columns = ["p", "q", "w"]
    df.loc[df["p"] >= df["q"], ["p", "q"]] = df.loc[df["p"] >= df["q"], ["q", "p"]].values
    df["p+q"] = df['p'].astype(str) + "_" + df['q'].astype(str)

    # All edges
    all_edges = set(df["p+q"])
    all_nodes = set(df["p"]).union(set(df["q"]))

    neg_set = compute_neg_set(all_edges, len(all_nodes), len(all_edges))
    pos_set = list(df.loc[:, ["p", "q"]].to_records(index = False))
    
    labels  = [0] * len(neg_set) + [1] * len(pos_set)
    features = construct_hadamard(neg_set + pos_set, emb.shape[1])
    
    # Shuffle
    shuffle    = np.random.permutation(len(labels))
    labels   = np.array(labels)[shuffle]
    features = features[shuffle]
    
    # Create a and fit classifier
    clf     = LogisticRegression(random_state = 0).fit(features, labels)
    return clf
    
    
    

### Training N2VEC embeddings

In [11]:
from sklearn.metrics import average_precision_score
import json
import numpy as np
import pandas as pd

def construct_hadamard(edgelists, emb, n_dim):
    features = np.zeros((len(edgelists), n_dim))
    for i, (p, q) in enumerate(edgelists):
        e1 = emb[p]
        e2 = emb[q]
        features[i] = e1 * e2
    return features

def filter_train(test_df, trained_df, nodemap):
    trained_df.loc[trained_df[0] > trained_df[1], [0, 1]] = trained_df.loc[trained_df[0] > trained_df[1], [1, 0]].values
    trained_df["p+q"] = trained_df[0] + "_" + trained_df[1]
    t_edges_set       = set(trained_df["p+q"])
    test_df_f         = test_df[~test_df["p+q"].isin(t_edges_set)]
    test_df_a = test_df_f.replace({"node1": nodemap, "node2": nodemap})
    return test_df_a


In [12]:
test_df = pd.read_csv(test_file, sep  = "\t")
test_df["p+q"] = test_df["node1"] + "_" + test_df["node2"]

for network in networks:
    print(f"For Network {network}:")
    df  = pd.read_csv(f"{output_folder}/n2vec_annotated_{network}.tsv", 
                  sep = "\t", 
                  header = None)
    print(f"\t Getting Embedding...")
    emb = np.load(f"{output_folder}/n2vec_{network}_emb_dim_300.npy")
    print(f"\t Using LR classifier and training")
    clf = construct_and_train_model(df, emb)
    
    # Network Nodemap
    with open(f"{output_folder}/n2vec_{network}.json", "r") as oj:
        nodemap = json.load(oj)
    
    # Train dataframe
    trained_df = pd.read_csv(f"{output_folder}/{network}.tsv", 
                         header = None, 
                         sep = "\t")
    
    test_df_a  = filter_train(test_df, trained_df, nodemap)
    edges    = list(test_df_a.loc[:, ["node1", "node2"]].to_records(index = False))

    features = construct_hadamard(edges, emb, 300)
    print(f"Performing Prediction...")
    results  = clf.predict_proba(features)
    p_score  = average_precision_score(test_df_a["True_score"], results[:, 1])
    print(f"\t ==> Average Precision Score: {p_score}\n\n")
    test_df_a["n2vec_prediction"] = results[:, 1]
    test_df_a.to_csv(f"{output_folder}/n2vec_{network}_n2vec_predictions_300.tsv", sep = "\t", index = False)

For Network network_0.2:
	 Getting Embedding...
	 Using LR classifier and training
Performing Prediction...
	 ==> Average Precision Score: 0.5749500295818181


For Network network_0.4:
	 Getting Embedding...
	 Using LR classifier and training
Performing Prediction...
	 ==> Average Precision Score: 0.6647023774785192


For Network network_0.6:
	 Getting Embedding...
	 Using LR classifier and training
Performing Prediction...
	 ==> Average Precision Score: 0.7213897691631074


For Network network_0.8:
	 Getting Embedding...
	 Using LR classifier and training
Performing Prediction...
	 ==> Average Precision Score: 0.6819148985068374


